In [1]:
import os, csv, pandas as pd, logging as lg
from mysql.connector import *

In [3]:
os.chdir(r"C:\Users\Suryansh Grover\Downloads\datasets")

In [4]:
os.listdir()

['FitBit data.csv']

In [5]:
class Log:
    def __init__(self):
        try:
            self.logFile="Fitbit.log"
            
            #removing the log file if already exists so as not to congest it.
            if os.path.exists(self.logFile):
                os.remove(self.logFile)
            lg.basicConfig(filename=self.logFile, level=lg.INFO, format="%(asctime)s %(levelname)s %(message)s")
            
            #Adding the StreamHandler to record logs in the console.
            self.console_log = lg.StreamHandler()
            self.console_log.setLevel(lg.INFO) #setting level to the console log.
            self.format = lg.Formatter("%(asctime)s %(message)s")
            self.console_log.setFormatter(self.format) #defining format for the console log.
            lg.getLogger('').addHandler(self.console_log) #adding handler to the console log.
        
        except Exception as e:
            lg.info(e)
            
        else:
            lg.info("Log Class successfully executed!")

In [7]:
#our RAW DATASET.
raw = 'FitBit data.csv'

class CleanDump():
    def __init__(self):
        try:
            self.mydb = connect(host = "localhost", user = "root", passwd = "auntmay", use_pure = True) 
            self.cur = self.mydb.cursor()
            
        except Exception as e:
            lg.error(e)
        else:
            lg.info("Connection is established to MySQL!")
            
    def __fetch_col(self): #using abstraction, creating a private function
        try:
            global raw
            with open(raw, "r") as data:
                data = csv.reader(data)
                col = ['']
                for row in data:
                    for i in row:
                        col[0] += f"{i} VARCHAR(200), "
                    return col[0][:-2] #so as to avoid the ", " at the last index.
        except Exception as e:
            lg.error(e)
            
    def use_db(self, db_name = "junkie"):
        try:
            self.cur.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
            self.cur.execute(f"USE {db_name}")
        except Exception as e:
            lg.error(e)
        else:
            lg.info(f"You are now using the database: {db_name}")
            
    def create_table(self, table_name="fitbit_data"):
        global raw
        try:
            self.cur.execute(f"DROP TABLE IF EXISTS {table_name}")
            self.cur.execute(f"CREATE TABLE {table_name} ({self.__fetch_col()})")
        except Exception as e:
            lg.error(e)
        else:
            lg.info("Your table is created!")
            return pd.read_sql(f"SELECT * FROM {table_name}", self.mydb)
        
    def insert_rows(self, table_name="fitbit_data" ):
        try:
            global raw
            r_i = 1 #record index
            
            with open(raw, 'r') as data:
                next(data)
                data = csv.reader(data, delimiter=',')
                
                for row in data:
                    insRow = [""] #Will hold the rows in an insertable form.
                    for i in row:
                        insRow[0]+=f'"{i}", '
                        
                    x = insRow[0][:-2]
                    lg.info(x)
                    self.cur.execute(f"INSERT INTO {table_name} VALUES({x})")
                    lg.info(f"Record {r_i} inserted!")
                    lg.info("-------------------------")
                    r_i+=1
                    
        except Exception as e:
            lg.error(e)   
        else:
            self.mydb.commit()
            lg.info("ALL RECORDS INSERTED!")
        
    def display_table(self, table_name = "fitbit_data"):
        try:
            lg.info("Your table is about to be displayed..")
            return pd.read_sql(f"SELECT * FROM {table_name}", self.mydb)
        except Exception as e:
                lg.error(e)
        

In [8]:
Log()
cd=CleanDump()

2022-07-08 11:48:46,755 Log Class successfully executed!
2022-07-08 11:48:46,988 Connection is established to MySQL!


In [9]:
cd._CleanDump__fetch_col()

'Id VARCHAR(200), ActivityDate VARCHAR(200), TotalSteps VARCHAR(200), TotalDistance VARCHAR(200), TrackerDistance VARCHAR(200), LoggedActivitiesDistance VARCHAR(200), VeryActiveDistance VARCHAR(200), ModeratelyActiveDistance VARCHAR(200), LightActiveDistance VARCHAR(200), SedentaryActiveDistance VARCHAR(200), VeryActiveMinutes VARCHAR(200), FairlyActiveMinutes VARCHAR(200), LightlyActiveMinutes VARCHAR(200), SedentaryMinutes VARCHAR(200), Calories VARCHAR(200)'

In [10]:
cd.use_db()

2022-07-08 11:49:03,255 You are now using the database: junkie


In [11]:
cd.create_table()

2022-07-08 11:49:12,594 Your table is created!


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories


In [12]:
cd.insert_rows()

2022-07-08 11:49:24,217 "1503960366", "3/25/2016", "11004", "7.1100001335144", "7.1100001335144", "0", "2.5699999332428", "0.46000000834465", "4.07000017166138", "0", "33", "12", "205", "804", "1819"
2022-07-08 11:49:24,232 Record 1 inserted!
2022-07-08 11:49:24,233 -------------------------
2022-07-08 11:49:24,234 "1503960366", "3/26/2016", "17609", "11.5500001907349", "11.5500001907349", "0", "6.92000007629395", "0.730000019073486", "3.91000008583069", "0", "89", "17", "274", "588", "2154"
2022-07-08 11:49:24,239 Record 2 inserted!
2022-07-08 11:49:24,240 -------------------------
2022-07-08 11:49:24,241 "1503960366", "3/27/2016", "12736", "8.52999973297119", "8.52999973297119", "0", "4.65999984741211", "0.159999996423721", "3.71000003814697", "0", "56", "5", "268", "605", "1944"
2022-07-08 11:49:24,244 Record 3 inserted!
2022-07-08 11:49:24,245 -------------------------
2022-07-08 11:49:24,247 "1503960366", "3/28/2016", "13231", "8.93000030517578", "8.93000030517578", "0", "3.190000

2022-07-08 11:49:24,366 Record 29 inserted!
2022-07-08 11:49:24,367 -------------------------
2022-07-08 11:49:24,368 "1624580081", "4/4/2016", "18464", "12", "12", "0", "0", "0", "12", "0", "0", "0", "270", "1170", "1574"
2022-07-08 11:49:24,369 Record 30 inserted!
2022-07-08 11:49:24,370 -------------------------
2022-07-08 11:49:24,370 "1624580081", "4/5/2016", "1335", "0.870000004768372", "0.870000004768372", "0", "0", "0", "0.870000004768372", "0", "0", "0", "74", "1366", "1308"
2022-07-08 11:49:24,372 Record 31 inserted!
2022-07-08 11:49:24,373 -------------------------
2022-07-08 11:49:24,374 "1624580081", "4/6/2016", "1004", "0.649999976158142", "0.649999976158142", "0", "0", "0", "0.649999976158142", "0", "0", "0", "55", "1385", "1276"
2022-07-08 11:49:24,376 Record 32 inserted!
2022-07-08 11:49:24,377 -------------------------
2022-07-08 11:49:24,377 "1624580081", "4/7/2016", "1111", "0.720000028610229", "0.720000028610229", "0", "0", "0", "0.720000028610229", "0", "0", "0", 

2022-07-08 11:49:24,482 "1844505072", "4/12/2016", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "426", "399"
2022-07-08 11:49:24,483 Record 60 inserted!
2022-07-08 11:49:24,484 -------------------------
2022-07-08 11:49:24,485 "1927972279", "4/1/2016", "4317", "2.99000000953674", "2.99000000953674", "0", "0", "0.370000004768372", "2.61999988555908", "0", "0", "11", "192", "854", "2590"
2022-07-08 11:49:24,487 Record 61 inserted!
2022-07-08 11:49:24,488 -------------------------
2022-07-08 11:49:24,489 "1927972279", "4/2/2016", "5662", "3.92000007629395", "3.92000007629395", "0", "0", "0", "3.92000007629395", "0", "0", "0", "267", "858", "2783"
2022-07-08 11:49:24,490 Record 62 inserted!
2022-07-08 11:49:24,491 -------------------------
2022-07-08 11:49:24,492 "1927972279", "4/3/2016", "3198", "2.21000003814697", "2.21000003814697", "0", "0", "0", "2.21000003814697", "0", "0", "0", "146", "1183", "2449"
2022-07-08 11:49:24,493 Record 63 inserted!
2022-07-08 11:49:24,494 ------

2022-07-08 11:49:24,587 Record 90 inserted!
2022-07-08 11:49:24,588 -------------------------
2022-07-08 11:49:24,589 "2026352035", "4/7/2016", "2767", "1.72000002861023", "1.72000002861023", "0", "0", "0", "1.72000002861023", "0", "0", "0", "170", "673", "1392"
2022-07-08 11:49:24,591 Record 91 inserted!
2022-07-08 11:49:24,591 -------------------------
2022-07-08 11:49:24,592 "2026352035", "4/8/2016", "3762", "2.32999992370605", "2.32999992370605", "0", "0", "0", "2.32999992370605", "0", "0", "0", "173", "719", "1425"
2022-07-08 11:49:24,595 Record 92 inserted!
2022-07-08 11:49:24,595 -------------------------
2022-07-08 11:49:24,596 "2026352035", "4/9/2016", "3098", "1.91999995708466", "1.91999995708466", "0", "0", "0", "1.91999995708466", "0", "0", "0", "185", "700", "1413"
2022-07-08 11:49:24,597 Record 93 inserted!
2022-07-08 11:49:24,598 -------------------------
2022-07-08 11:49:24,599 "2026352035", "4/10/2016", "5142", "3.19000005722046", "3.19000005722046", "0", "0", "0", "3.

2022-07-08 11:49:24,692 Record 121 inserted!
2022-07-08 11:49:24,693 -------------------------
2022-07-08 11:49:24,694 "2347167796", "4/11/2016", "10001", "6.6100001335144", "6.6100001335144", "0", "0.330000013113022", "2.9300000667572", "3.35999989509583", "0", "5", "52", "196", "788", "2009"
2022-07-08 11:49:24,696 Record 122 inserted!
2022-07-08 11:49:24,696 -------------------------
2022-07-08 11:49:24,697 "2347167796", "4/12/2016", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "425", "399"
2022-07-08 11:49:24,700 Record 123 inserted!
2022-07-08 11:49:24,701 -------------------------
2022-07-08 11:49:24,702 "2873212765", "4/1/2016", "890", "0.600000023841858", "0.600000023841858", "0", "0", "0", "0.600000023841858", "0", "0", "0", "41", "1399", "1335"
2022-07-08 11:49:24,704 Record 124 inserted!
2022-07-08 11:49:24,705 -------------------------
2022-07-08 11:49:24,706 "2873212765", "4/2/2016", "2214", "1.49000000953674", "1.49000000953674", "0", "0", "0", "1.49000000953674

...

2022-07-08 11:49:25,466 -------------------------
2022-07-08 11:49:25,468 "6391747486", "4/4/2016", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1440", "1820"
2022-07-08 11:49:25,471 Record 338 inserted!
2022-07-08 11:49:25,472 -------------------------
2022-07-08 11:49:25,472 "6391747486", "4/5/2016", "9766", "7.51000022888184", "0.779999971389771", "6.72705698013306", "0", "0.0500000007450581", "7.46000003814697", "0", "0", "6", "301", "1133", "2442"
2022-07-08 11:49:25,474 Record 339 inserted!
2022-07-08 11:49:25,475 -------------------------
2022-07-08 11:49:25,476 "6391747486", "4/6/2016", "165", "0.129999995231628", "0.129999995231628", "0", "0", "0", "0.129999995231628", "0", "0", "0", "8", "1432", "1849"
2022-07-08 11:49:25,478 Record 340 inserted!
2022-07-08 11:49:25,479 -------------------------
2022-07-08 11:49:25,480 "6391747486", "4/7/2016", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "1440", "1822"
2022-07-08 11:49:25,481 Record 341 inserted!
2022-07

2022-07-08 11:49:25,575 Record 367 inserted!
2022-07-08 11:49:25,576 -------------------------
2022-07-08 11:49:25,577 "7007744171", "4/2/2016", "10976", "7.34000015258789", "7.34000015258789", "0", "0.550000011920929", "0.670000016689301", "6.13000011444092", "0", "9", "16", "271", "1075", "2575"
2022-07-08 11:49:25,579 Record 368 inserted!
2022-07-08 11:49:25,580 -------------------------
2022-07-08 11:49:25,581 "7007744171", "4/3/2016", "16806", "11.2399997711182", "11.2399997711182", "0", "6.40999984741211", "1.37000000476837", "3.47000002861023", "0", "104", "31", "230", "1002", "3086"
2022-07-08 11:49:25,583 Record 369 inserted!
2022-07-08 11:49:25,584 -------------------------
2022-07-08 11:49:25,584 "7007744171", "4/4/2016", "15266", "11.3999996185303", "10.210000038147", "5.18984985351563", "5.80000019073486", "0.759999990463257", "4.78999996185303", "0.0500000007450581", "70", "21", "292", "1057", "2886"
2022-07-08 11:49:25,586 Record 370 inserted!
2022-07-08 11:49:25,587 ---

2022-07-08 11:49:25,670 Record 395 inserted!
2022-07-08 11:49:25,671 -------------------------
2022-07-08 11:49:25,672 "8053475328", "4/7/2016", "17078", "13.1999998092651", "13.1999998092651", "0", "9.97000026702881", "0.239999994635582", "2.99000000953674", "0", "101", "6", "190", "1143", "3176"
2022-07-08 11:49:25,673 Record 396 inserted!
2022-07-08 11:49:25,674 -------------------------
2022-07-08 11:49:25,675 "8053475328", "4/8/2016", "11693", "9.53999996185303", "9.53999996185303", "0", "6.76999998092651", "0.259999990463257", "2.5", "0", "60", "6", "153", "1221", "2790"
2022-07-08 11:49:25,676 Record 397 inserted!
2022-07-08 11:49:25,677 -------------------------
2022-07-08 11:49:25,678 "8053475328", "4/9/2016", "11159", "9.14000034332275", "9.14000034332275", "0", "6.69000005722046", "0.25", "2.19000005722046", "0", "59", "6", "120", "1255", "2715"
2022-07-08 11:49:25,680 Record 398 inserted!
2022-07-08 11:49:25,681 -------------------------
2022-07-08 11:49:25,682 "8053475328"

2022-07-08 11:49:25,771 Record 425 inserted!
2022-07-08 11:49:25,772 -------------------------
2022-07-08 11:49:25,772 "8583815059", "4/1/2016", "2174", "1.70000004768372", "1.70000004768372", "0", "0", "0", "1.70000004768372", "0", "0", "0", "112", "1328", "2360"
2022-07-08 11:49:25,775 Record 426 inserted!
2022-07-08 11:49:25,776 -------------------------
2022-07-08 11:49:25,776 "8583815059", "4/2/2016", "3494", "2.73000001907349", "2.73000001907349", "0", "0", "0", "2.69000005722046", "0", "0", "0", "166", "1274", "2517"
2022-07-08 11:49:25,779 Record 427 inserted!
2022-07-08 11:49:25,779 -------------------------
2022-07-08 11:49:25,780 "8583815059", "4/3/2016", "2380", "1.86000001430511", "1.86000001430511", "0", "0", "0", "1.86000001430511", "0", "0", "0", "131", "1309", "2400"
2022-07-08 11:49:25,782 Record 428 inserted!
2022-07-08 11:49:25,782 -------------------------
2022-07-08 11:49:25,783 "8583815059", "4/4/2016", "3159", "2.46000003814697", "2.46000003814697", "0", "0", "0

2022-07-08 11:49:25,872 Record 454 inserted!
2022-07-08 11:49:25,873 -------------------------
2022-07-08 11:49:25,873 "8877689391", "4/10/2016", "28497", "27.5300006866455", "27.5300006866455", "0", "21.9200000762939", "1.12000000476837", "4.46000003814697", "0", "128", "46", "211", "1055", "4526"
2022-07-08 11:49:25,875 Record 455 inserted!
2022-07-08 11:49:25,875 -------------------------
2022-07-08 11:49:25,876 "8877689391", "4/11/2016", "10622", "8.0600004196167", "8.0600004196167", "0", "1.47000002861023", "0.150000005960464", "6.36999988555908", "0.00999999977648258", "18", "7", "225", "1190", "2820"
2022-07-08 11:49:25,878 Record 456 inserted!
2022-07-08 11:49:25,879 -------------------------
2022-07-08 11:49:25,880 "8877689391", "4/12/2016", "2350", "1.77999997138977", "1.77999997138977", "0", "0", "0", "1.77999997138977", "0", "0", "0", "58", "531", "938"
2022-07-08 11:49:25,881 Record 457 inserted!
2022-07-08 11:49:25,883 -------------------------
2022-07-08 11:49:25,892 ALL

In [13]:
cd.display_table()

2022-07-08 11:49:47,348 Your table is about to be displayed..


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.1100001335144,7.1100001335144,0,2.5699999332428,0.46000000834465,4.07000017166138,0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.5500001907349,11.5500001907349,0,6.92000007629395,0.730000019073486,3.91000008583069,0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.52999973297119,8.52999973297119,0,4.65999984741211,0.159999996423721,3.71000003814697,0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93000030517578,8.93000030517578,0,3.19000005722046,0.790000021457672,4.94999980926514,0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.84999990463257,7.84999990463257,0,2.16000008583069,1.0900000333786,4.6100001335144,0,28,28,243,763,1886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,8877689391,4/8/2016,23014,20.3899993896484,20.3899993896484,0,11.1000003814697,0.629999995231628,8.61999988555908,0,70,29,359,982,4196
453,8877689391,4/9/2016,16470,8.06999969482422,8.06999969482422,0,0,0.0199999995529652,8.02000045776367,0,90,9,289,1052,3841
454,8877689391,4/10/2016,28497,27.5300006866455,27.5300006866455,0,21.9200000762939,1.12000000476837,4.46000003814697,0,128,46,211,1055,4526
455,8877689391,4/11/2016,10622,8.0600004196167,8.0600004196167,0,1.47000002861023,0.150000005960464,6.36999988555908,0.00999999977648258,18,7,225,1190,2820
